# Sistema de Limpieza y Enriquecimiento de Direcciones

Este notebook demuestra cómo usar el nuevo sistema de limpieza y enriquecimiento de datos de direcciones que se integra con nuestra herramienta de geocodificación reversa.

## Funcionalidades implementadas:
1. **Sistema de Completado Inteligente de Direcciones**
2. **Validador y Normalizador de Direcciones**
3. **Sistema de Enriquecimiento de Datos Geográficos**

## Requisitos:
- Archivo `.env` con tu API key de OpenCage
- Dependencias instaladas (ejecutar primera celda)

In [1]:
# Instalar dependencias si no están disponibles
import subprocess
import sys

packages = ['python-dotenv', 'opencage', 'pandas']

for package in packages:
    try:
        __import__(package.replace('-', '_'))
        print(f"✓ {package} ya está instalado")
    except ImportError:
        print(f"Instalando {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} instalado")

Instalando python-dotenv...
✓ python-dotenv instalado
✓ opencage ya está instalado
✓ python-dotenv instalado
✓ opencage ya está instalado
✓ pandas ya está instalado
✓ pandas ya está instalado


In [2]:
# Importar el sistema de enriquecimiento
import sys
import os

# Añadir el directorio del batch processor al path
batch_processor_path = os.path.join(os.getcwd(), 'reverse_geocoding_batch_processor')
if batch_processor_path not in sys.path:
    sys.path.append(batch_processor_path)

from address_enhancer import AddressEnhancer
import pandas as pd

print("✓ Sistema de enriquecimiento de direcciones importado correctamente")

✓ Sistema de enriquecimiento de direcciones importado correctamente


In [3]:
# Inicializar el enhancer
enhancer = AddressEnhancer()
print("✓ AddressEnhancer inicializado correctamente")

✓ AddressEnhancer inicializado correctamente


## 1. Detectar Direcciones Incompletas

Vamos a probar la funcionalidad de detección de direcciones incompletas:

In [4]:
# Ejemplos de direcciones con diferentes niveles de completitud
test_addresses = [
    "Calle 123",  # Incompleta - falta ciudad
    "123 Main Street, Madrid",  # Más completa
    "Av. Insurgentes Sur 1234, Colonia del Valle, Ciudad de México, 03100",  # Completa
    "Madrid",  # Muy incompleta - solo ciudad
    "",  # Vacía
    "c/ Gran Vía 25, Madrid, 28013"  # Completa con abreviación
]

print("Análisis de completitud de direcciones:\n")
for i, address in enumerate(test_addresses, 1):
    analysis = enhancer.detect_incomplete_address(address)
    print(f"{i}. Dirección: '{address}'")
    print(f"   Completa: {analysis['is_complete']}")
    print(f"   Confianza: {analysis['confidence']:.2f}")
    print(f"   Componentes faltantes: {analysis['missing_components']}")
    # print(f"   Componentes encontrados: {analysis['components_found']}")
    print()

Análisis de completitud de direcciones:

1. Dirección: 'Calle 123'
   Completa: False
   Confianza: 0.50
   Componentes faltantes: ['city', 'postal_code']

2. Dirección: '123 Main Street, Madrid'
   Completa: False
   Confianza: 0.75
   Componentes faltantes: ['postal_code']

3. Dirección: 'Av. Insurgentes Sur 1234, Colonia del Valle, Ciudad de México, 03100'
   Completa: True
   Confianza: 1.00
   Componentes faltantes: []

4. Dirección: 'Madrid'
   Completa: False
   Confianza: 0.00
   Componentes faltantes: ['street_number', 'street_name', 'city', 'postal_code']

5. Dirección: ''
   Completa: False
   Confianza: 0.00
   Componentes faltantes: ['address']

6. Dirección: 'c/ Gran Vía 25, Madrid, 28013'
   Completa: True
   Confianza: 1.00
   Componentes faltantes: []



## 2. Normalización de Direcciones

Probemos la normalización de formatos de direcciones:

In [5]:
# Direcciones con diferentes formatos y abreviaciones
addresses_to_normalize = [
    "c/ gran via 25, madrid",
    "av. insurgentes sur 1234",
    "pl. mayor 15",
    "cr 15 # 123-45",
    "123 main st, new york",
    "456 oak ave, los angeles"
]

print("Normalización de direcciones:\n")
for address in addresses_to_normalize:
    normalized_es = enhancer.normalize_address_format(address, 'es')
    normalized_en = enhancer.normalize_address_format(address, 'en')
    
    print(f"Original: '{address}'")
    print(f"Normalizada (ES): '{normalized_es}'")
    print(f"Normalizada (EN): '{normalized_en}'")
    print()

Normalización de direcciones:

Original: 'c/ gran via 25, madrid'
Normalizada (ES): 'C/ Gran Via 25, Madrid'
Normalizada (EN): 'C/ Gran Via 25, Madrid'

Original: 'av. insurgentes sur 1234'
Normalizada (ES): 'Avenida. Insurgentes Sur 1234'
Normalizada (EN): 'Avenue. Insurgentes Sur 1234'

Original: 'pl. mayor 15'
Normalizada (ES): 'Plaza. Mayor 15'
Normalizada (EN): 'Place. Mayor 15'

Original: 'cr 15 # 123-45'
Normalizada (ES): 'Carrera 15 # 123-45'
Normalizada (EN): 'Cr 15 # 123-45'

Original: '123 main st, new york'
Normalizada (ES): '123 Main St, New York'
Normalizada (EN): '123 Main Street, New York'

Original: '456 oak ave, los angeles'
Normalizada (ES): '456 Oak Ave, Los Angeles'
Normalizada (EN): '456 Oak Avenue, Los Angeles'



## 3. Sugerencias de Corrección

Probemos las sugerencias de corrección para errores comunes:

In [6]:
# Direcciones con errores de tipeo
addresses_with_errors = [
    "callle mayor 25",  # 'callle' debería ser 'calle'
    "avendia reforma 123",  # 'avendia' debería ser 'avenida'
    "plasa de armas 15",  # 'plasa' debería ser 'plaza'
    "123 main stret",  # 'stret' debería ser 'street'
    "456 oak avenu"  # 'avenu' debería ser 'avenue'
]

print("Sugerencias de corrección:\n")
for address in addresses_with_errors:
    suggestions = enhancer.suggest_address_corrections(address)
    print(f"Original: '{address}'")
    print(f"Sugerencias: {suggestions}")
    print()

Sugerencias de corrección:

Original: 'callle mayor 25'
Sugerencias: ['Calle Mayor 25', 'Callle Mayor 25']

Original: 'avendia reforma 123'
Sugerencias: ['Avenida Reforma 123', 'Avendia Reforma 123']

Original: 'plasa de armas 15'
Sugerencias: ['Plaza De Armas 15', 'Plasa De Armas 15']

Original: '123 main stret'
Sugerencias: ['123 Main Street', '123 Main Stret']

Original: '456 oak avenu'
Sugerencias: ['456 Oak Avenue', '456 Oak Avenu']



## 4. Completado Inteligente de Direcciones

Probemos el completado de direcciones usando coordenadas y geocodificación:

In [7]:
# Ejemplo 1: Completar usando coordenadas (geocodificación inversa)
print("=== Completado usando coordenadas ===\n")

partial_address = "Calle 123"
coordinates = (40.4168, -3.7038)  # Madrid, España

result = enhancer.complete_address(partial_address, coordinates, language='es')

print(f"Dirección parcial: '{result['original_address']}'")
print(f"Dirección completada: '{result['completed_address']}'")
print(f"Método usado: {result['method_used']}")
print(f"Confianza: {result['confidence']:.2f}")
print(f"Componentes encontrados: {result['components']}")
print()

=== Completado usando coordenadas ===

Dirección parcial: 'Calle 123'
Dirección completada: 'Puerta del Sol, 7, 28013 Madrid, España'
Método usado: reverse_geocoding
Confianza: 0.90
Componentes encontrados: {'ISO_3166-1_alpha-2': 'ES', 'ISO_3166-1_alpha-3': 'ESP', 'ISO_3166-2': ['ES-MD'], '_category': 'building', '_normalized_city': 'Madrid', '_type': 'building', 'city': 'Madrid', 'city_district': 'Centro', 'continent': 'Europe', 'country': 'España', 'country_code': 'es', 'house_number': '7', 'neighbourhood': 'Barrio de los Austrias', 'political_union': 'European Union', 'postcode': '28013', 'quarter': 'Sol', 'road': 'Puerta del Sol', 'state': 'Comunidad de Madrid', 'state_code': 'MD'}

Dirección parcial: 'Calle 123'
Dirección completada: 'Puerta del Sol, 7, 28013 Madrid, España'
Método usado: reverse_geocoding
Confianza: 0.90
Componentes encontrados: {'ISO_3166-1_alpha-2': 'ES', 'ISO_3166-1_alpha-3': 'ESP', 'ISO_3166-2': ['ES-MD'], '_category': 'building', '_normalized_city': 'Madrid'

In [8]:
# Ejemplo 2: Completar usando geocodificación directa
print("=== Completado usando geocodificación directa ===\n")

partial_address = "Gran Vía, Madrid"

result = enhancer.complete_address(partial_address, language='es')

print(f"Dirección parcial: '{result['original_address']}'")
print(f"Dirección completada: '{result['completed_address']}'")
print(f"Método usado: {result['method_used']}")
print(f"Confianza: {result['confidence']:.2f}")
if 'coordinates' in result:
    print(f"Coordenadas obtenidas: {result['coordinates']}")
print(f"Sugerencias: {result['suggestions']}")
print()

=== Completado usando geocodificación directa ===

Dirección parcial: 'Gran Vía, Madrid'
Dirección completada: 'Gran Vía, 28013 Madrid, España'
Método usado: forward_geocoding
Confianza: 0.80
Coordenadas obtenidas: {'lat': 40.4220457, 'lng': -3.7087216}
Sugerencias: []

Dirección parcial: 'Gran Vía, Madrid'
Dirección completada: 'Gran Vía, 28013 Madrid, España'
Método usado: forward_geocoding
Confianza: 0.80
Coordenadas obtenidas: {'lat': 40.4220457, 'lng': -3.7087216}
Sugerencias: []



## 6. Validación de Códigos Postales

Probemos la validación de códigos postales:

## 5. Limpieza de Caracteres Especiales

Una nueva funcionalidad integrada desde el sistema original: la limpieza de caracteres especiales en direcciones. Esto es útil para sistemas que no manejan bien Unicode o necesitan compatibilidad con sistemas legacy.

In [9]:
# Importar la función de limpieza
from reverse_geocoding_batch_processor.address_enhancer import clean_address_text

# Ejemplos de direcciones con caracteres especiales
addresses_with_special_chars = [
    "Calle Mayor 25 (Madrid) - España!!!",
    "Av. Insurgentes Sur #1234 @Colonia Roma",
    "123 Main St. <New York> {USA}",
    "c/ Gran Vía 15 [Centro] ~Madrid~",
    "Rúa José García López, 42 - Coruña/España",
    "Café República 123 São Paulo/Brasil"
]

print("=== Limpieza Conservadora (preserva acentos) ===\n")
for address in addresses_with_special_chars:
    cleaned = clean_address_text(address, aggressive=False)
    print(f"Original:  '{address}'")
    print(f"Limpia:    '{cleaned}'")
    print()

print("\n=== Limpieza Agresiva (elimina acentos) ===\n")
for address in addresses_with_special_chars:
    cleaned = clean_address_text(address, aggressive=True)
    print(f"Original:  '{address}'")
    print(f"Limpia:    '{cleaned}'")
    print()

=== Limpieza Conservadora (preserva acentos) ===

Original:  'Calle Mayor 25 (Madrid) - España!!!'
Limpia:    'Calle Mayor 25 Madrid - España'

Original:  'Av. Insurgentes Sur #1234 @Colonia Roma'
Limpia:    'Av. Insurgentes Sur 1234 Colonia Roma'

Original:  '123 Main St. <New York> {USA}'
Limpia:    '123 Main St. New York USA'

Original:  'c/ Gran Vía 15 [Centro] ~Madrid~'
Limpia:    'c Gran Vía 15 Centro Madrid'

Original:  'Rúa José García López, 42 - Coruña/España'
Limpia:    'Rúa José García López, 42 - CoruñaEspaña'

Original:  'Café República 123 São Paulo/Brasil'
Limpia:    'Café República 123 São PauloBrasil'


=== Limpieza Agresiva (elimina acentos) ===

Original:  'Calle Mayor 25 (Madrid) - España!!!'
Limpia:    'Calle Mayor 25 Madrid - Espana'

Original:  'Av. Insurgentes Sur #1234 @Colonia Roma'
Limpia:    'Av. Insurgentes Sur 1234 Colonia Roma'

Original:  '123 Main St. <New York> {USA}'
Limpia:    '123 Main St. New York USA'

Original:  'c/ Gran Vía 15 [Centro] ~Madrid~

In [10]:
# Demostrar completado de direcciones con limpieza de caracteres especiales
print("=== Completado de Direcciones CON Limpieza ===\n")

# Dirección parcial con caracteres especiales problemáticos
partial_address_special = "Gran Vía!!! (Madrid)"
coordinates = (40.4200, -3.7025)  # Madrid, España

# Completar SIN limpieza
result_no_clean = enhancer.complete_address(
    partial_address_special, 
    coordinates, 
    language='es', 
    clean_special_chars=False
)

print("SIN limpieza de caracteres especiales:")
print(f"  Dirección completada: '{result_no_clean['completed_address']}'")
print()

# Completar CON limpieza conservadora
result_clean = enhancer.complete_address(
    partial_address_special, 
    coordinates, 
    language='es', 
    clean_special_chars=True,
    aggressive=False
)

print("CON limpieza conservadora:")
print(f"  Dirección completada: '{result_clean['completed_address']}'")
print()

# Completar CON limpieza agresiva
result_aggressive = enhancer.complete_address(
    partial_address_special, 
    coordinates, 
    language='es', 
    clean_special_chars=True,
    aggressive=True
)

print("CON limpieza agresiva:")
print(f"  Dirección completada: '{result_aggressive['completed_address']}'")
print()

# Comparar componentes limpiados
if 'components' in result_clean:
    print("Componentes con limpieza conservadora:")
    for key, value in result_clean['components'].items():
        if value:
            print(f"  {key}: '{value}'")
    print()

if 'components' in result_aggressive:
    print("Componentes con limpieza agresiva:")
    for key, value in result_aggressive['components'].items():
        if value:
            print(f"  {key}: '{value}'")

=== Completado de Direcciones CON Limpieza ===

SIN limpieza de caracteres especiales:
  Dirección completada: 'Calle Gran Via, 25, 28013 Madrid, España'

SIN limpieza de caracteres especiales:
  Dirección completada: 'Calle Gran Via, 25, 28013 Madrid, España'

CON limpieza conservadora:
  Dirección completada: 'Calle Gran Via, 25, 28013 Madrid, España'

CON limpieza conservadora:
  Dirección completada: 'Calle Gran Via, 25, 28013 Madrid, España'

CON limpieza agresiva:
  Dirección completada: 'Calle Gran Via, 25, 28013 Madrid, Espana'

Componentes con limpieza conservadora:
  ISO_3166-1_alpha-2: 'ES'
  ISO_3166-1_alpha-3: 'ESP'
  ISO_3166-2: '['ES-MD', 'ES-M']'
  _category: 'building'
  _normalized_city: 'Madrid'
  _type: 'building'
  city: 'Madrid'
  continent: 'Europe'
  country: 'España'
  country_code: 'es'
  county: 'Madrid'
  county_code: 'M'
  house_number: '25'
  political_union: 'European Union'
  postcode: '28013'
  road: 'Calle Gran Via'
  state: 'Comunidad de Madrid'
  sta

In [11]:
# Códigos postales de diferentes países
postal_codes_test = [
    ('28013', 'ES'),  # España - válido
    ('12345', 'US'),  # Estados Unidos - válido
    ('K1A 0A9', 'CA'),  # Canadá - válido
    ('12345-6789', 'US'),  # Estados Unidos ZIP+4 - válido
    ('123', 'ES'),  # España - inválido (muy corto)
    ('ABC123', None),  # Sin país - intentar detectar
    ('', 'ES'),  # Vacío
]

print("Validación de códigos postales:\n")
for postal_code, country in postal_codes_test:
    validation = enhancer.validate_postal_code(postal_code, country)
    print(f"Código: '{postal_code}' (País: {country})")
    print(f"Válido: {validation['is_valid']}")
    if 'reason' in validation:
        print(f"Razón: {validation['reason']}")
    if 'possible_country' in validation:
        print(f"País posible: {validation['possible_country']}")
    print()

Validación de códigos postales:

Código: '28013' (País: ES)
Válido: True

Código: '12345' (País: US)
Válido: True

Código: 'K1A 0A9' (País: CA)
Válido: True

Código: '12345-6789' (País: US)
Válido: True

Código: '123' (País: ES)
Válido: False

Código: 'ABC123' (País: None)
Válido: False
Razón: no_pattern_match

Código: '' (País: ES)
Válido: False
Razón: empty_postal_code



## 7. Enriquecimiento de Datos Geográficos

Probemos el enriquecimiento completo de datos de ubicación:

In [12]:
# Datos básicos de dirección para enriquecer
basic_address_data = {
    'address': 'Puerta del Sol, Madrid, España'
}

print("Enriqueciendo datos de ubicación...\n")
enriched = enhancer.enrich_location_data(basic_address_data)

# Mostrar resultados de forma organizada
print("=== DATOS ENRIQUECIDOS ===")
print(f"Dirección original: {enriched.get('address', 'N/A')}")

if 'coordinates' in enriched:
    print(f"\n📍 Coordenadas:")
    print(f"   Latitud: {enriched['coordinates']['lat']}")
    print(f"   Longitud: {enriched['coordinates']['lng']}")

if 'timezone' in enriched:
    print(f"\n🕐 Zona horaria:")
    print(f"   Nombre: {enriched['timezone']['name']}")
    print(f"   Offset: {enriched['timezone']['offset_string']}")

if 'administrative_levels' in enriched:
    admin = enriched['administrative_levels']
    print(f"\n🏛️ Niveles administrativos:")
    print(f"   País: {admin['country']} ({admin['country_code']})")
    print(f"   Estado/Provincia: {admin['state']}")
    print(f"   Ciudad: {admin['city']}")
    print(f"   Barrio: {admin['neighbourhood']}")

if 'geographic_info' in enriched:
    geo = enriched['geographic_info']
    print(f"\n🌍 Información geográfica:")
    print(f"   Código postal: {geo['postcode']}")
    print(f"   Continente: {geo['continent']}")
    print(f"   Moneda: {geo.get('currency', {}).get('name', 'N/A')}")
    print(f"   Código telefónico: +{geo['calling_code']}")
    print(f"   Geohash: {geo['geohash']}")

print(f"\n⏰ Procesado en: {enriched.get('enrichment_timestamp', 'N/A')}")

Enriqueciendo datos de ubicación...

=== DATOS ENRIQUECIDOS ===
Dirección original: Puerta del Sol, Madrid, España

📍 Coordenadas:
   Latitud: 40.416863
   Longitud: -3.7038762

🕐 Zona horaria:
   Nombre: Europe/Madrid
   Offset: +0200

🏛️ Niveles administrativos:
   País: Spain (es)
   Estado/Provincia: Madrid
   Ciudad: Madrid
   Barrio: Barrio de los Austrias

🌍 Información geográfica:
   Código postal: 28013
   Continente: 
   Moneda: Euro
   Código telefónico: +34
   Geohash: ezjmgtwgx303brpt3u2c

⏰ Procesado en: 2025-09-04T14:25:56.923917
=== DATOS ENRIQUECIDOS ===
Dirección original: Puerta del Sol, Madrid, España

📍 Coordenadas:
   Latitud: 40.416863
   Longitud: -3.7038762

🕐 Zona horaria:
   Nombre: Europe/Madrid
   Offset: +0200

🏛️ Niveles administrativos:
   País: Spain (es)
   Estado/Provincia: Madrid
   Ciudad: Madrid
   Barrio: Barrio de los Austrias

🌍 Información geográfica:
   Código postal: 28013
   Continente: 
   Moneda: Euro
   Código telefónico: +34
   Geohash: 

## 8. Procesamiento en Lote

Vamos a crear un archivo de ejemplo y procesarlo en lote:

In [13]:
# Crear datos de ejemplo
sample_data = [
    {'address': 'c/ gran via 25', 'lat': 40.4200, 'lng': -3.7025},
    {'address': 'Sagrada Familia, Barcelona'},
    {'address': 'plasa mayor, madrid'},
    {'address': 'Times Square', 'city': 'New York'},
    {'address': 'callle reforma 123', 'country': 'Mexico'}
]

# Crear DataFrame
df_sample = pd.DataFrame(sample_data)
print("Datos de ejemplo:")
print(df_sample)
print()

# Guardar como CSV temporal
sample_file = 'sample_addresses_temp.csv'
df_sample.to_csv(sample_file, index=False)
print(f"Datos guardados en {sample_file}")

Datos de ejemplo:
                      address    lat     lng      city country
0              c/ gran via 25  40.42 -3.7025       NaN     NaN
1  Sagrada Familia, Barcelona    NaN     NaN       NaN     NaN
2         plasa mayor, madrid    NaN     NaN       NaN     NaN
3                Times Square    NaN     NaN  New York     NaN
4          callle reforma 123    NaN     NaN       NaN  Mexico

Datos guardados en sample_addresses_temp.csv


In [14]:
# Procesar en lote
print("Procesando direcciones en lote...\\n")

# Convertir a lista de diccionarios
addresses_to_process = df_sample.to_dict('records')

# Procesar (usando delay corto para demo)
enhanced_addresses = enhancer.process_address_batch(
    addresses_to_process, 
    delay=0.5,  # Delay corto para la demo
    language='es',
    clean_special_chars=True,  # 🆕 Activar limpieza de caracteres especiales
    aggressive=False  # 🆕 Usar limpieza conservadora (preservar acentos)
)

print("\\n=== RESULTADOS DEL PROCESAMIENTO EN LOTE ===\\n")
print("✨ Nuevo: Procesamiento incluye limpieza de caracteres especiales")

Procesando direcciones en lote...\n
Procesando dirección 1/5
Procesando dirección 2/5
Procesando dirección 2/5
Procesando dirección 3/5
Procesando dirección 3/5
Procesando dirección 4/5
Procesando dirección 4/5
Procesando dirección 5/5
\n=== RESULTADOS DEL PROCESAMIENTO EN LOTE ===\n
✨ Nuevo: Procesamiento incluye limpieza de caracteres especiales
Procesando dirección 5/5
\n=== RESULTADOS DEL PROCESAMIENTO EN LOTE ===\n
✨ Nuevo: Procesamiento incluye limpieza de caracteres especiales


In [15]:
# Mostrar resultados del procesamiento
for i, addr in enumerate(enhanced_addresses, 1):
    print(f"=== Dirección {i} ===")
    print(f"Original: {addr.get('address', 'N/A')}")
    print(f"Completada: {addr.get('completed_address', 'N/A')}")
    print(f"Normalizada: {addr.get('normalized_address', 'N/A')}")
    print(f"Método usado: {addr.get('method_used', 'N/A')}")
    
    if 'quality_metrics' in addr:
        metrics = addr['quality_metrics']
        print(f"Puntuación de completitud: {metrics['completeness_score']:.2f}")
        print(f"Tiene coordenadas: {metrics['has_coordinates']}")
    
    if 'coordinates' in addr:
        print(f"Coordenadas: ({addr['coordinates']['lat']:.4f}, {addr['coordinates']['lng']:.4f})")
    
    if 'administrative_levels' in addr:
        admin = addr['administrative_levels']
        print(f"País: {admin.get('country', 'N/A')}")
        print(f"Ciudad: {admin.get('city', 'N/A')}")
    
    print()

=== Dirección 1 ===
Original: c/ gran via 25
Completada: Calle Gran Via, 25, 28013 Madrid, España
Normalizada: Calle Gran Via, 25, 28013 Madrid, España
Método usado: reverse_geocoding
Puntuación de completitud: 0.50
Tiene coordenadas: True
País: Spain
Ciudad: Madrid

=== Dirección 2 ===
Original: Sagrada Familia, Barcelona
Completada: Sagrada Familia, Barcelona
Normalizada: Sagrada Familia, Barcelona
Método usado: error
Puntuación de completitud: 0.25
Tiene coordenadas: True

=== Dirección 3 ===
Original: plasa mayor, madrid
Completada: plasa mayor, madrid
Normalizada: Plasa Mayor, Madrid
Método usado: error
Puntuación de completitud: 0.25
Tiene coordenadas: True

=== Dirección 4 ===
Original: Times Square
Completada: Times Square
Normalizada: Times Square
Método usado: error
Puntuación de completitud: 0.00
Tiene coordenadas: True

=== Dirección 5 ===
Original: callle reforma 123
Completada: callle reforma 123
Normalizada: Callle Reforma 123
Método usado: error
Puntuación de completitu

In [16]:
# Demostrar diferentes opciones de limpieza
test_data_special = [
    {'address': 'c/ gran via 25 (madrid)!!! @españa'},
    {'address': 'av. insurgentes #1234 <méxico>'},
    {'address': 'rúa josé garcía lópez 42 ~coruña~'}
]

print("=== COMPARACIÓN DE OPCIONES DE LIMPIEZA ===\\n")

# Procesar SIN limpieza
print("1. SIN limpieza:")
result_no_clean = enhancer.process_address_batch(
    test_data_special.copy(), 
    delay=0.3, 
    language='es',
    clean_special_chars=False
)
for i, addr in enumerate(result_no_clean, 1):
    print(f"   {i}. '{addr.get('completed_address', addr.get('address', 'N/A'))}'")

print("\\n2. CON limpieza conservadora:")
result_conservative = enhancer.process_address_batch(
    test_data_special.copy(), 
    delay=0.3, 
    language='es',
    clean_special_chars=True,
    aggressive=False
)
for i, addr in enumerate(result_conservative, 1):
    print(f"   {i}. '{addr.get('completed_address', addr.get('address', 'N/A'))}'")

print("\\n3. CON limpieza agresiva:")
result_aggressive = enhancer.process_address_batch(
    test_data_special.copy(), 
    delay=0.3, 
    language='es',
    clean_special_chars=True,
    aggressive=True
)
for i, addr in enumerate(result_aggressive, 1):
    print(f"   {i}. '{addr.get('completed_address', addr.get('address', 'N/A'))}'")

print("\\n🔍 Observa cómo los caracteres especiales (!, @, <>, ~) son removidos")
print("🔍 En modo agresivo, también se eliminan los acentos (ú → u)")

=== COMPARACIÓN DE OPCIONES DE LIMPIEZA ===\n
1. SIN limpieza:
Procesando dirección 1/3
Procesando dirección 2/3
Procesando dirección 2/3
Procesando dirección 3/3
Procesando dirección 3/3
   1. 'Palazzo, Calle Gran Vía, 25, 28220 Majadahonda, España'
   2. 'Calle Insurgentes Sur 1234, Los Reyes, 36570 Irapuato, GUA, México'
   3. 'La Coruña, Galicia, España'
\n2. CON limpieza conservadora:
Procesando dirección 1/3
   1. 'Palazzo, Calle Gran Vía, 25, 28220 Majadahonda, España'
   2. 'Calle Insurgentes Sur 1234, Los Reyes, 36570 Irapuato, GUA, México'
   3. 'La Coruña, Galicia, España'
\n2. CON limpieza conservadora:
Procesando dirección 1/3
Procesando dirección 2/3
Procesando dirección 2/3
Procesando dirección 3/3
Procesando dirección 3/3
   1. 'Palazzo, Calle Gran Vía, 25, 28220 Majadahonda, España'
   2. 'Calle Insurgentes Sur 1234, Los Reyes, 36570 Irapuato, GUA, México'
   3. 'La Coruña, Galicia, España'
\n3. CON limpieza agresiva:
Procesando dirección 1/3
   1. 'Palazzo, Calle Gran

In [17]:
# Exportar resultados
output_file = 'enhanced_addresses_demo.csv'
enhancer.export_enhanced_addresses(enhanced_addresses, output_file, 'csv')

print(f"✓ Resultados exportados a {output_file}")

# Mostrar algunas estadísticas
df_results = pd.read_csv(output_file)
print(f"\nEstadísticas del procesamiento:")
print(f"Total de direcciones procesadas: {len(df_results)}")
print(f"Columnas generadas: {len(df_results.columns)}")
print(f"\nPrimeras columnas: {list(df_results.columns[:10])}")

Direcciones enriquecidas exportadas a enhanced_addresses_demo.csv
✓ Resultados exportados a enhanced_addresses_demo.csv

Estadísticas del procesamiento:
Total de direcciones procesadas: 5
Columnas generadas: 67

Primeras columnas: ['address', 'lat', 'lng', 'city', 'country', 'original_address', 'completed_address', 'method_used', 'confidence', 'components_ISO_3166-1_alpha-2']


In [18]:
# Limpiar archivos temporales
import os

try:
    os.remove(sample_file)
    print(f"Archivo temporal {sample_file} eliminado")
except:
    pass

print("\n🎉 ¡Demo completada! El sistema de limpieza y enriquecimiento está funcionando correctamente.")

Archivo temporal sample_addresses_temp.csv eliminado

🎉 ¡Demo completada! El sistema de limpieza y enriquecimiento está funcionando correctamente.


## Integración con el Sistema Existente

El nuevo sistema se integra perfectamente con tu herramienta de geocodificación reversa existente:

1. **Usa las mismas dependencias** (python-dotenv, opencage, pandas)
2. **Comparte la misma API key** de OpenCage
3. **Extiende las funcionalidades** sin romper el código existente
4. **Mantiene compatibilidad** con los formatos de datos actuales
5. **🆕 Incluye limpieza de caracteres especiales** como en el sistema original

### Uso desde línea de comandos:
```bash
# Básico
python reverse_geocoding_batch_processor/address_enhancer.py input_addresses.csv -o enhanced_output.csv --language es

# Con limpieza de caracteres especiales
python reverse_geocoding_batch_processor/address_enhancer.py input_addresses.csv -o enhanced_output.csv --language es --clean

# Con limpieza agresiva (elimina también acentos)
python reverse_geocoding_batch_processor/address_enhancer.py input_addresses.csv -o enhanced_output.csv --language es --clean --aggressive
```

### Uso programático:
```python
from address_enhancer import AddressEnhancer

enhancer = AddressEnhancer()

# Completado básico
result = enhancer.complete_address("calle incompleta", coordinates=(lat, lng))

# Completado con limpieza conservadora
result = enhancer.complete_address(
    "calle!!! incompleta@", 
    coordinates=(lat, lng),
    clean_special_chars=True,
    aggressive=False
)

# Completado con limpieza agresiva
result = enhancer.complete_address(
    "calle!!! incomplétá@", 
    coordinates=(lat, lng),
    clean_special_chars=True,
    aggressive=True  # "incomplétá" se convierte en "incompleta"
)

# Procesamiento en lote con limpieza
enhanced = enhancer.process_address_batch(
    addresses, 
    clean_special_chars=True,
    aggressive=False
)
```

### 🆕 Nuevas Funcionalidades de Limpieza:

| Opción | Descripción | Ejemplo |
|--------|-------------|---------|
| `clean_special_chars=False` | Sin limpieza (por defecto) | `"Calle Mayor!!! @Madrid"` → Sin cambios |
| `clean_special_chars=True, aggressive=False` | Limpieza conservadora | `"Calle Mayor!!! @Madrid"` → `"Calle Mayor Madrid"` |
| `clean_special_chars=True, aggressive=True` | Limpieza agresiva | `"Calle Alcalá!!! @Madrid"` → `"Calle Alcala Madrid"` |

### Caracteres que se eliminan:
- **Conservadora**: `? ¿ ! ¡ @ # $ % ^ & * ( ) _ + = < > { } [ ] | \\ / : ; " ' ` ~`
- **Agresiva**: Los anteriores + acentos (`á é í ó ú à è ì ò ù ä ë ï ö ü â ê î ô û ã ẽ ĩ õ ũ ñ ç ß`)

### ⚙️ Integración con el Sistema Original:
Esta funcionalidad está basada en la función `clean_address_text()` del sistema original `reverse_geocoding_batch.py`, manteniendo la misma lógica y comportamiento para garantizar consistencia entre las herramientas.